In [3]:
import os
from typing import Optional, Union
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
def join_csv(search_path, max_tables:Optional[int]=None) -> pd.DataFrame:
    out_table = pd.DataFrame()
    if max_tables is None: max_tables = np.inf
    num_tables = 0
    for root, dirs, files in os.walk(search_path, topdown=True):
        dirs.sort(reverse=True)
        for name in files:
            if num_tables > max_tables : return out_table
            table_path = os.path.join(root, name)
            current_table = pd.read_csv(table_path)
            
            if out_table.shape[0] == 0: 
                out_table = current_table
                continue
            
            out_table = pd.join(current_table, out_table, on=)
            
            num_tables += 1
        return out_table

In [57]:
join_csv('E:\KickStarter\input')

E:\KickStarter\input\Kickstarter_2020-11\Kickstarter.csv


pog
